In [32]:
#!/usr/bin/env python

# convert gml to json and add community

# import libraries
import argparse
import networkx as nx
from networkx.readwrite import json_graph
from modularity_maximization import partition
from modularity_maximization.utils import get_modularity

#def graphmltojson(graphfile, outfile):
    # """
    # Converts GraphML file to json while adding communities/modularity groups
    # using python-louvain. JSON output is usable with D3 force layout.
    # Usage:
    # python convert.py -i mygraph.graphml -o outfile.json
    # """

G = nx.read_gml('../src/data/influencer/TheDataFox.gml')

#comm_dict = partition(G)
# for key, value in comm_dict.iteritems():
#     print key, value
nx.set_node_attributes(G, 'group_modularity', comm_dict.values)
# for comm in set(comm_dict.values()):
#     print("Community %d"%comm)
#     print(', '.join([node for node in comm_dict if comm_dict[node] == comm]))

# print('Modularity of such partition for network is %.3f' %\
#       get_modularity(G, comm_dict))

# # adds partition/community number as attribute named 'modularitygroup'
# for n, d in G.nodes
#     d['modularitygroup'] = comm_dict[n]

# node_link = json_graph.node_link_data(G)
# json = json_graph.dumps(node_link)

# #Write to file
# fo = open('output.json', "w")
# fo.write(json)
# fo.close()

#
# if __name__ == '__main__':
#     parser = argparse.ArgumentParser(description='Convert from GraphML to json. ')
#     parser.add_argument('-i', '--input', help='Input file name (graphml)', required=True)
#     parser.add_argument('-o', '--output', help='Output file name/path', required=True)
#     args = parser.parse_args()
#     graphmltojson(args.input, args.output)

# graphmltojson('influencer\TheDataFox.gml', 'influencer\TheDataFox.json')




TypeError: unhashable type: 'dict'

In [1]:
import tweepy
import time
import os
import sys
import json
import argparse
from datetime import datetime

In [6]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit_notify=True)

print("Connected to Twitter!")

Connected to Twitter!


In [9]:
# Create the directories we need
if not os.path.exists(FOLLOWING_DIR):
    os.makedirs(FOLLOWING_DIR)

if not os.path.exists(USER_DIR):
    os.makedirs(USER_DIR)

In [10]:
enc = lambda x: x.encode('ascii', errors='ignore')

In [11]:
def get_follower_ids(centre, max_depth=1, current_depth=0, taboo_list=[]):

    if current_depth == max_depth:
        print('out of depth')
        return taboo_list

    if centre in taboo_list:
        # we've been here before
        print('Already been here.')
        return taboo_list
    else:
        taboo_list.append(centre)

    try:
        userfname = os.path.join(USER_DIR, str(centre) + '.json')
        if not os.path.exists(userfname):
            print('Retrieving user details for twitter id %s' % str(centre))
            while True:
                try:
                    user = api.get_user(centre)

                    d = {'name': user.name,
                         'screen_name': user.screen_name,
                         'profile_image_url' : user.profile_image_url,
                         'created_at' : str(user.created_at),
                         'id': user.id,
                         'friends_count': user.friends_count,
                         'followers_count': user.followers_count,
                         'followers_ids': user.followers_ids()}

                    with open(userfname, 'w') as outf:
                        outf.write(json.dumps(d, indent=1))

                    user = d
                    break
                except tweepy.TweepError as error:
                    print(type(error))

                    if str(error) == 'Not authorized.':
                        print('Can''t access user data - not authorized.')
                        return taboo_list

                    if str(error) == 'User has been suspended.':
                        print('User suspended.')
                        return taboo_list

                    errorObj = error[0][0]

                    print(errorObj)

                    if errorObj['message'] == 'Rate limit exceeded':
                        print('Rate limited. Sleeping for 15 minutes.')
                        time.sleep(15 * 60 + 15)
                        continue

                    return taboo_list
        else:
            user = json.loads(file(userfname).read())

        screen_name = enc(user['screen_name'])
        fname = os.path.join(FOLLOWING_DIR, screen_name + '.csv')
        friendids = []

        if not os.path.exists(fname):
            print('No cached data for screen name "%s"' % screen_name)
            with open(fname, 'w') as outf:
                params = (enc(user['name']), screen_name)
                print('Retrieving friends for user "%s" (%s)' % params)

                # page over friends
                c = list(tweepy.Cursor(api.friends, id=user['id']).items())

                friend_count = 0
                while True:
                    try:
                        friend = next(c)
                        friendids.append(friend.id)
                        params = (friend.id, enc(friend.screen_name), enc(friend.name))
                        outf.write('%s\t%s\t%s\n' % params)
                        friend_count += 1
                        if friend_count >= MAX_FRIENDS:
                            print('Reached max no. of friends for "%s".' % friend.screen_name)
                            break
                    except tweepy.TweepError:
                        # hit rate limit, sleep for 15 minutes
                        print('Rate limited. Sleeping for 15 minutes.')
                        time.sleep(15 * 60 + 15)
                        continue
                    except StopIteration:
                        break
        else:
            friendids = [int(line.strip().split('\t')[0]) for line in file(fname)]

        print('Found %d friends for %s' % (len(friendids), screen_name))

        # get friends of friends
        cd = current_depth
        if cd+1 < max_depth:
            for fid in friendids[:FRIENDS_OF_FRIENDS_LIMIT]:
                taboo_list = get_follower_ids(fid, max_depth=max_depth,
                    current_depth=cd+1, taboo_list=taboo_list)

        if cd+1 < max_depth and len(friendids) > FRIENDS_OF_FRIENDS_LIMIT:
            print('Not all friends retrieved for %s.' % screen_name)

    except Exception as error:
        print('Error retrieving followers for user id: ', centre)
        print(error)

        if os.path.exists(fname):
            os.remove(fname)
            print('Removed file "%s".' % fname)

        sys.exit(1)

    return taboo_list

In [12]:
if __name__ == '__main__':
    ap = argparse.ArgumentParser()
    ap.add_argument("-s", "--screen-name", required=True, help="Screen name of twitter user")
    ap.add_argument("-d", "--depth", required=True, type=int, help="How far to follow user network")
    args = vars(ap.parse_args())

    twitter_screenname = args['screen_name']
    depth = int(args['depth'])

    if depth < 1 or depth > 5:
        print('Depth value %d is not valid. Valid range is 1-5.' % depth)
        sys.exit('Invalid depth argument.')

    print('Max Depth: %d' % depth)
    matches = api.lookup_users(screen_names=[twitter_screenname])

    if len(matches) == 1:
        print(get_follower_ids(matches[0].id, max_depth=depth))
    else:
        print('Sorry, could not find twitter user with screen name: %s' % twitter_screenname)


usage: __main__.py [-h] -s SCREEN_NAME -d DEPTH
__main__.py: error: the following arguments are required: -s/--screen-name, -d/--depth


SystemExit: 2

C:\Python36\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
